In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# parameter
encoding_key = '%2FkoG7UseaGiJgZLma7NKZqmx9vTCYK%2FhgSoWp5KHOspAiRubeUPeChwwOZ0A4mhQ%2BsXM3vETx2srDT1uxCajJQ%3D%3D'
numrow = 500
pageNo = 1

In [2]:
def xmltodf(contents):
    root = ET.fromstring(contents)
    data_list = []
    
    for item in root.findall('.//item'):
        data = {}
        for col in item.findall('./col'):
            name = col.get('name')
            value = col.text
            data[name] = value
        data_list.append(data)
    df = pd.DataFrame(data_list)
    return df

In [3]:
def people_api(encoding_key, page, perPage):
    url = 'https://api.odcloud.kr/api/15097972/v1/uddi:a7891af9-05d8-4a11-bffb-c20d3d713601?'
    page = 'page=' + str(page)
    perPage = '&perPage=' + str(perPage)
    returnType = '&returnType=XML'
    serviceKey = '&serviceKey=' + encoding_key
    total_url = url + page + perPage + returnType + serviceKey

    response = requests.get(total_url)
    contents = response.text
    df = xmltodf(contents)
    
    columns_to_delete = ["기준연월", "행정기관코드", "계", "남자", "여자"]
    df = df.drop(columns=columns_to_delete)
    
    age_columns = [f'만{i}세남자' for i in range(110)] + ['만110세이상남자'] + [f'만{i}세여자' for i in range(110)] + ['만110세이상여자']

    for column in age_columns:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    
    age_ranges = [(0, 9), (10, 19), (20, 29), (30, 39), (40, 49), (50, 59), (60, 69), (70, 79), (80, 89), (90, 99)]

    # 각 연령대에 대한 남자 데이터를 해당 컬럼에 할당
    for start_age, end_age in age_ranges:
        column_name = f'{start_age}대남자'
        df[column_name] = 0  # 초기값 설정
        for age in range(start_age, end_age + 1):
            df[column_name] += df[f'만{age}세남자']

        column_name = f'{start_age}대여자'
        df[column_name] = 0  # 초기값 설정
        for age in range(start_age, end_age + 1):
            df[column_name] += df[f'만{age}세여자']
    
    df = df.drop(columns=age_columns)
    
    return df

In [14]:
def total_people(encoding_key, pageNo, numrow):
    total_df = pd.DataFrame()
    while True:
        df = people_api(encoding_key, pageNo, numrow)
        df_len = len(df)
        total_df = pd.concat([total_df, df])
        if (df_len < 500):
            break
        else:
            pageNo += 1

    # '시도명' : '지역명'
    total_df = total_df.rename(columns={'시도명': '지역명'})
    
    total_df['지역명'] = total_df['지역명'].replace({
        '서울특별시': '서울',
        '부산광역시': '부산',
        '대구광역시': '대구',
        '인천광역시': '인천',
        '광주광역시': '광주',
        '대전광역시': '대전',
        '울산광역시': '울산',
        '세종특별자치시': '세종',
        '경기도': '경기',
        '강원도': '강원',
        '충청북도': '충북',
        '충청남도': '충남',
        '전라북도': '전북',
        '전라남도': '전남',
        '경상북도': '경북',
        '경상남도': '경남',
        '제주특별자치도': '제주'
    })
    
    total_df['총인구'] = total_df.filter(like='대').sum(axis=1)

    # 읍면동 개수
    total_df['읍면동 개수'] = total_df.groupby('시군구명')['읍면동명'].transform('nunique')
    
    sorted_columns = ['지역명', '시군구명', '읍면동명', '읍면동 개수', '0대남자', '0대여자', '10대남자', '10대여자', '20대남자', '20대여자', '30대남자', '30대여자',
            '40대남자', '40대여자', '50대남자', '50대여자', '60대남자', '60대여자', '70대남자', '70대여자', '80대남자', '80대여자', '90대남자', '90대여자', '총인구']
    
    total_df = total_df[sorted_columns]
    
    return total_df

total_df = total_people(encoding_key, pageNo, numrow)




C:\Users\DCU\AppData\Local\Temp\ipykernel_10896\4242276727.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = 0  # 초기값 설정
C:\Users\DCU\AppData\Local\Temp\ipykernel_10896\4242276727.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = 0  # 초기값 설정
C:\Users\DCU\AppData\Local\Temp\ipykernel_10896\4242276727.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once 

In [17]:
total_df.to_excel("연령분포.xlsx")